In [1]:
import sys
sys.path.append('classif_experim')
import pandas as pd
import numpy as np
from classif_experim.hf_skelarn_wrapper import SklearnTransformerClassif
from settings import TEST_DATASET_EN, TEST_DATASET_ES
import torch

/home/elural/miniconda3/envs/pan-clef-2024/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_predictions_task_4(lan):
    df_path = TEST_DATASET_EN if lan == "en" else TEST_DATASET_ES

    test_df = pd.read_json(df_path, orient="records")
    ids = test_df["id"].values

    preds_array = np.array([])
    for fold_id in range(5):
        model = SklearnTransformerClassif.load(f"TASK4_BEST_MODEL_{lan}_FOLD_{fold_id}", "cuda")
        preds = model.predict(test_df["text"].values, [])
        preds = preds.reshape(1000, 1)
        preds_array = np.hstack((preds_array, preds)) if preds_array.size else preds
        del model
        torch.cuda.empty_cache()
    
    voting = (np.sum(preds_array, axis=1) > 3).astype(float)
    return pd.DataFrame(
        {
            "id": ids,
            "category": voting
        }
    )

In [3]:
df = make_predictions_task_4("en")
df.to_csv("task4_en_elias&sergio.csv", index=False)

In [4]:
df = make_predictions_task_4("es")
df.to_csv("task4_es_elias&sergio.csv", index=False)